# How to deal with complex/large Documents

이전 노트북에서는 조직에서 흔히 볼 수 있는 다양한 유형의 파일과 데이터 형식에 대한 솔루션을 개발했으며, 이는 대부분의 사용 사례를 다룹니다. 하지만 복잡한 파일에서 답을 찾아야 하는 질문을 처리할 때 문제가 있다는 것을 알게 될 것입니다. 이러한 파일의 복잡성은 파일의 길이와 그 안에 정보가 분산되어 있는 방식에서 비롯됩니다. 대용량 문서는 검색 엔진에게 항상 어려운 과제입니다.

이러한 복잡한 파일의 한 예로 수백 페이지에 달하고 이미지, 표, 양식 등의 형태로 정보를 포함하는 기술 사양 가이드나 제품 설명서를 들 수 있습니다. 책 역시 그 길이와 이미지나 표의 존재로 인해 복잡합니다.

이러한 파일은 일반적으로 PDF 형식으로 되어 있습니다. 이러한 PDF를 더 잘 처리하려면 각 문서를 특별한 소스로 취급하고 페이지 단위(1페이지=1청크)로 처리하는 더 스마트한 파싱 방법이 필요합니다. 목표는 시스템에서 더 정확하고 빠른 답변을 얻는 것입니다. 다행히도 조직에는 일반적으로 이러한 유형의 문서가 많지 않으므로 예외를 만들어 다르게 처리할 수 있습니다.

예를 들어 사용 사례가 PDF인 경우 PyPDF 라이브러리 (https://pypi.org/project/pypdf/) 또는 Azure AI Document Intelligence SDK(이전 Form Recognizer) (https://learn.microsoft.com/en-us/azure/ai-services/document-intelligence/overview?view=doc-intel-3.0.0) 를 사용하고 OpenAI API를 사용하여 벡터화하여 콘텐츠를 벡터 기반 인덱스로 푸시할 수 있습니다. 이것이 아마도 가장 간단하고 빠른 방법일 것입니다. 그러나 사용 사례에 데이터 레이크, Sharepoint 라이브러리 또는 여러 파일 형식의 수천 개의 문서가 있고 동적으로 변경될 수 있는 기타 문서 데이터 소스에 연결해야 하는 경우에는 Azure 검색 엔진, 노트북 1-2의 수집 및 문서 크래킹 및 AI 강화 기능을 사용하여 많은 사용자 지정 코드를 피하고 싶을 것입니다.

In [ ]:
import os
import json
import time
import requests
import random
from collections import OrderedDict
import urllib.request
from tqdm import tqdm
from typing import List

from langchain_openai import AzureOpenAIEmbeddings
from langchain_openai import AzureChatOpenAI
from langchain_core.retrievers import BaseRetriever
from langchain_core.callbacks import CallbackManagerForRetrieverRun
from langchain_core.documents import Document
from langchain_core.messages import HumanMessage
from langchain_core.runnables import ConfigurableField
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter


from common.utils import parse_pdf, read_pdf_files, text_to_base64
from common.prompts import DOCSEARCH_PROMPT
from common.utils import CustomAzureSearchRetriever


from IPython.display import Markdown, HTML, display  

from dotenv import load_dotenv
load_dotenv("credentials.env")

def printmd(string):
    display(Markdown(string))
    
os.makedirs("data/techdocs/",exist_ok=True)

# Name of the container in your Blob Storage Datasource ( in credentials.env) 
BLOB_CONTAINER_NAME = "techdocs"

c = os.environ["BLOB_CONNECTION_STRING"].split(";")
BASE_CONTAINER_URL = c[0].split("=")[1] + BLOB_CONTAINER_NAME + "/"

LOCAL_FOLDER = "./data/techdocs/"

os.makedirs(LOCAL_FOLDER,exist_ok=True)

In [ ]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]

In [ ]:
embedder = AzureOpenAIEmbeddings(deployment=os.environ["EMBEDDING_DEPLOYMENT_NAME"], chunk_size=1) 

## 1 - Manual Document Cracking with Push to Vector-based Index

Azure Service에 대한 PDF 기술 문서를 보관하는 techdocs 라는 이름의 컨테이너가 있습니다. 이 모든 책의 페이지가 담긴 cogsrch-index-techdocs 인덱스를 생성하여 로드해 보겠습니다.

먼저 이 책들을 로컬 머신에 다운로드합니다. 여기에서는 빠른 테스트를 위해 각 서비스의 Overview 섹션만을 인덱싱합니다. 전체 문서를 인덱싱하고저 하는 경우 아래 파일명을 변경하세요.

In [ ]:
techdocs = ["Azure_Cognitive_Search_Documentation_Overview.pdf", "Azure_AI_services_Document_Intelligence_Documentation_Overview.pdf"]

Let's download the files to the local `./data/` folder:

In [ ]:
# Blob에 있는 PDF를 Local로 다운로드하는 부분

for book in tqdm(techdocs):
    book_url = BASE_CONTAINER_URL + book + os.environ['BLOB_SAS_TOKEN']
    urllib.request.urlretrieve(book_url, LOCAL_FOLDER+ book)

### What to use: pyPDF or AI Documment Intelligence API (Form Recognizer)?

'utils.py'에는 **parse_pdf()** 함수가 있습니다. 이 유틸리티 함수는 PyPDF 라이브러리를 사용하여 로컬 파일을 구문 분석할 수 있으며, Azure AI Document Intelligence(이전 양식 인식기)를 사용하여 로컬 또는 from_url PDF 파일을 구문 분석할 수도 있습니다.

`form_recognizer=False`인 경우, 이 함수는 파이썬 pyPDF 라이브러리를 사용하여 PDF를 구문 분석하며, 75%의 경우 제대로 작동합니다.<br>

`form_recognizer=True`를 설정하는 것은 AI 문서 인텔리전스 API(이전의 양식 인식기)를 사용하는 가장 좋은(그리고 느린) 구문 분석 방법입니다. 사용할 사전 구축 모델을 지정할 수 있으며, 기본값은 `model="prebuilt-document"`입니다. 그러나 표, 차트 및 그림이 포함된 복잡한 문서가 있는 경우 다음을 사용해 볼 수 있습니다.
model="prebuilt-layout"`을 사용하면 각 페이지의 모든 뉘앙스를 포착할 수 있습니다(물론 시간이 더 오래 걸립니다).

**참고: 많은 PDF는 스캔 이미지입니다. 예를 들어, 스캔하여 PDF로 저장한 서명된 계약서는 pyPDF에서 파싱되지 않습니다. AI 문서 인텔리전스 API만 작동합니다.**

In [ ]:
book_pages_map = dict()
for book in techdocs:
    print("Extracting Text from",book,"...")
    
    # Capture the start time
    start_time = time.time()
    
    # Parse the PDF
    book_path = LOCAL_FOLDER+book
    book_map = parse_pdf(file=book_path, form_recognizer=True, model="prebuilt-document", from_url=False, verbose=True)
    book_pages_map[book]= book_map
    
    # Capture the end time and Calculate the elapsed time
    end_time = time.time()
    elapsed_time = end_time - start_time

    print(f"Parsing took: {elapsed_time:.6f} seconds")
    print(f"{book} contained {len(book_map)} pages\n")

Now let's check a random page of each book to make sure the parsing was done correctly:

In [ ]:
for bookname,bookmap in book_pages_map.items():
    print(bookname,"\n","chunk text:",bookmap[random.randint(0, 29)][2][:120],"...\n")

위에서 살펴본 바와 같이 Azure Document Intelligence는 pyPDF보다 우수하다는 것이 입증되었습니다. **프로덕션 시나리오의 경우 Azure Document Intelligence를 지속적으로 사용할 것을 강력히 권장합니다**. 이 경우 '미리 빌드된 문서', '미리 빌드된 레이아웃' 등의 사용 가능한 모델 중에서 현명한 선택을 하는 것이 중요합니다. 모델 선택에 대한 자세한 내용은 [여기](https://learn.microsoft.com/en-us/azure/ai-services/document-intelligence/choose-model-feature?view=doc-intel-3.0.0)에서 확인할 수 있습니다.


## Create Vector-based index


이제 책의 청크(각 책의 각 페이지)의 콘텐츠가 사전 `book_pages_map`에 있으므로 이 콘텐츠가 위치할 Azure 검색 엔진에 벡터 인덱스를 생성해 보겠습니다.

In [ ]:
book_index_name = "cogsrch-index-techdocs"

In [ ]:
### Create Azure Search Vector-based Index
# Setup the Payloads header
headers = {'Content-Type': 'application/json','api-key': os.environ['AZURE_SEARCH_KEY']}
params = {'api-version': os.environ['AZURE_SEARCH_API_VERSION']}

Index와 관련하여 다음 사항에 유의하세요.

- ParentKey(부모 키) 필드가 없습니다.
- page_num 필드가 존재합니다.

ParentKey 필드가 없는 것은 PULL 방식이 아닌 PUSH 방식을 활용하기 때문입니다. 이 접근 방식은 Azure AI 검색 엔진에서 제공하는 통합 인덱싱을 활용하지 않고 있음을 나타냅니다. 대신 구문 분석, OCR 수행, 벡터와 함께 콘텐츠를 수동으로 생성 및 푸시하는 프로세스에 참여하고 있습니다.

이 수동 구문 분석 프로세스에는 pyPDF 라이브러리 또는 Azure Document Intelligence API가 사용됩니다. 이러한 API를 사용하면 지정된 문자 수가 아닌 페이지별로 콘텐츠를 세분화할 수 있으며, 이는 Azure AI 검색 인덱서에서 사용하는 방법입니다. 따라서 인덱스에 page_num을 필드로 포함할 수 있습니다.


REST API 버전 2023-10-01-프리뷰는 외부 및 내부 벡터화를 지원합니다. 이 노트북은 외부 벡터화 전략을 가정합니다. 이 API는 또한

인덱싱 및 점수 매기기 위한 매개변수와 함께 vectorSearch 알고리즘, hnsw 및 exhaustiveKnn 최접근 이웃.
알고리즘 구성의 여러 조합을 위한 vectorProfiles.
벡터 검색 알고리즘에는 완전 k-최근접 이웃(KNN) 및 계층적 탐색 가능한 작은 세계(HNSW)가 포함됩니다. 완전 KNN은 전체 벡터 공간을 스캔하는 무차별 검색을 수행합니다. HNSW는 대략적인 최인접 이웃(ANN) 검색을 수행합니다. KNN은 높은 정확도로 정확한 최근접 이웃 검색 결과를 제공하지만, 계산 비용과 낮은 확장성으로 인해 대규모 데이터 세트나 실시간 애플리케이션에는 비현실적입니다. 반면, HNSW는 대략적인 가장 가까운 이웃을 빠르게 찾아내어 가장 가까운 이웃 검색을 위한 매우 효율적이고 확장 가능한 솔루션을 제공하므로 대규모 및 고차원 데이터 애플리케이션에 더 적합합니다.

벡터 구성에 대한 자세한 내용은 여기에서 확인하세요.
https://learn.microsoft.com/en-us/azure/search/vector-search-how-to-create-index?tabs=config-2023-10-01-Preview%2Crest-2023-11-01%2Cpush%2Cportal-check-index

In [ ]:
index_payload = {
    "name": book_index_name,
    "vectorSearch": {
        "algorithms": [  # We are showing here 3 types of search algorithms configurations that you can do
             {
                 "name": "my-hnsw-config-1",
                 "kind": "hnsw",
                 "hnswParameters": {
                     "m": 4,
                     "efConstruction": 400,
                     "efSearch": 500,
                     "metric": "cosine"
                 }
             },
             {
                 "name": "my-hnsw-config-2",
                 "kind": "hnsw",
                 "hnswParameters": {
                     "m": 8,
                     "efConstruction": 800,
                     "efSearch": 800,
                     "metric": "cosine"
                 }
             },
             {
                 "name": "my-eknn-config",
                 "kind": "exhaustiveKnn",
                 "exhaustiveKnnParameters": {
                     "metric": "cosine"
                 }
             }
        ],
        "vectorizers": [
            {
                "name": "openai",
                "kind": "azureOpenAI",
                "azureOpenAIParameters":
                {
                    "resourceUri" : os.environ['AZURE_OPENAI_ENDPOINT'],
                    "apiKey" : os.environ['AZURE_OPENAI_API_KEY'],
                    "deploymentId" : os.environ['EMBEDDING_DEPLOYMENT_NAME']
                }
            }
        ],
        "profiles": [  # profiles is the diferent kind of combinations of algos and vectorizers
            {
             "name": "my-vector-profile-1",
             "algorithm": "my-hnsw-config-1",
             "vectorizer":"openai"
            },
            {
             "name": "my-vector-profile-2",
             "algorithm": "my-hnsw-config-2",
             "vectorizer":"openai"
            },
            {
             "name": "my-vector-profile-3",
             "algorithm": "my-eknn-config",
             "vectorizer":"openai"
            }
        ]
    },
    "semantic": {
        "configurations": [
            {
                "name": "my-semantic-config",
                "prioritizedFields": {
                    "titleField": {
                        "fieldName": "title"
                    },
                    "prioritizedContentFields": [
                        {
                            "fieldName": "chunk"
                        }
                    ],
                    "prioritizedKeywordsFields": []
                }
            }
        ]
    },
    "fields": [
        {"name": "id", "type": "Edm.String", "key": "true", "filterable": "true" },
        {"name": "title","type": "Edm.String","searchable": "true","retrievable": "true"},
        {"name": "chunk","type": "Edm.String","searchable": "true","retrievable": "true"},
        {"name": "name", "type": "Edm.String", "searchable": "true", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},
        {"name": "location", "type": "Edm.String", "searchable": "false", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},
        {"name": "page_num","type": "Edm.Int32","searchable": "false","retrievable": "true"},
        {
            "name": "chunkVector",
            "type": "Collection(Edm.Single)",
            "dimensions": 1536,
            "vectorSearchProfile": "my-vector-profile-3", # we picked profile 3 to show that this index uses eKNN vs HNSW (on prior notebooks)
            "searchable": "true",
            "retrievable": "true",
            "filterable": "false",
            "sortable": "false",
            "facetable": "false"
        }
        
    ],
}

r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexes/" + book_index_name,
                 data=json.dumps(index_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)

In [ ]:
# Uncomment to debug errors
# r.text

## 문서 청크와 해당 벡터를 인덱스에 업로드


다음 코드는 각 도서의 각 청크를 반복하고 Azure Search Rest API 업로드 방법을 사용하여 각 문서를 해당 벡터(OpenAI 임베딩 모델 사용)와 함께 인덱스에 삽입합니다.

In [ ]:
%%time
for bookname,bookmap in book_pages_map.items():
    print("Uploading chunks from",bookname)
    for page in tqdm(bookmap):
        try:
            page_num = page[0] + 1
            content = page[2]
            book_url = BASE_CONTAINER_URL + bookname
            upload_payload = {
                "value": [
                    {
                        "id": text_to_base64(bookname + str(page_num)),
                        "title": f"{bookname}_page_{str(page_num)}",
                        "chunk": content,
                        "chunkVector": embedder.embed_query(content if content!="" else "-------"),
                        "name": bookname,
                        "location": book_url,
                        "page_num": page_num,
                        "@search.action": "upload"
                    },
                ]
            }

            r = requests.post(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexes/" + book_index_name + "/docs/index",
                                 data=json.dumps(upload_payload), headers=headers, params=params)
            if r.status_code != 200:
                print(r.status_code)
                print(r.text)
        except Exception as e:
            print("Exception:",e)
            print(content)
            continue

## Query the Index

In [ ]:
#QUESTION = "What's Azure AI Search?"
QUESTION = "Can I move, backup, and restore indexes?"

In [ ]:
indexes = [book_index_name]
k=10 # in this index k corresponds to the top pages as well

In [ ]:
retriever = CustomAzureSearchRetriever(indexes=[book_index_name], topK=k, reranker_threshold=1)

**Note**: 이 청크는 이전 노트북처럼 각각 5000자가 아니라 각 페이지가 청크이기 때문에 더 큰 k=20을 선택한다는 것을 알 수 있습니다.

In [ ]:
COMPLETION_TOKENS = 2500
llm = AzureChatOpenAI(deployment_name=os.environ["GPT35_DEPLOYMENT_NAME"], temperature=0.5, max_tokens=COMPLETION_TOKENS).configurable_alternatives(
    ConfigurableField(id="model"),
    default_key="gpt35",
    gpt4=AzureChatOpenAI(deployment_name=os.environ["GPT4_DEPLOYMENT_NAME"], temperature=0, max_tokens=COMPLETION_TOKENS),
)

In `utils.py` we created the **CustomAzureSearchRetriever** class that we will use going forward

In [ ]:
chain = (
    {
        "context": itemgetter("question") | retriever, # Passes the question to the retriever and the results are assign to context
        "question": itemgetter("question")
    }
    | DOCSEARCH_PROMPT  # Passes the 4 variables above to the prompt template
    | llm   # Passes the finished prompt to the LLM
    | StrOutputParser()  # converts the output (Runnable object) to the desired output (string)
)

#### With GPT 3.5

In [ ]:
for chunk in chain.with_config(configurable={"model": "gpt35"}).stream({"question": QUESTION}):
    print(chunk, end="", flush=True)

#### With GPT 4

In [ ]:
for chunk in chain.with_config(configurable={"model": "gpt4"}).stream(
    {"question": QUESTION, "language": "English"}):
    print(chunk, end="", flush=True)

# Summary

이 노트북에서는 [하이브리드 검색](https://learn.microsoft.com/en-us/azure/search/search-get-started-vector#hybrid-search)(텍스트 + 벡터 검색)을 사용하여 복잡하고 큰 문서를 처리하고 이를 Q&A에 사용할 수 있도록 하는 방법을 배웠습니다.


또한 Azure Document Intelligence API의 성능과 수동 문서 구문 분석이 필요한 프로덕션 시나리오(Azure Search 인덱서 문서 크래킹 대신)에 권장되는 이유에 대해 알아봤습니다.


벡터 기능 및 하이브리드 검색 기능과 함께 Azure AI Search를 사용하면 Weaviate, Qdrant, Milvus, Pinecone 등과 같은 다른 벡터 데이터베이스가 필요하지 않습니다.


# NEXT
지금까지 Azure AI Search에 저장된 문서에서 우수한 답변을 얻기 위해 OpenAI 벡터 및 완성 API를 사용하는 방법에 대해 알아봤습니다. 이것이 바로 `GPT 스마트 검색 엔진`의 근간이 됩니다.

하지만 뭔가 빠진 것이 있습니다: 바로 **이 엔진과 대화하는 방법**입니다.


다음 노트북에서는 **메모리**의 개념을 이해해 보겠습니다. 이는 사용자와 대화를 나눌 수 있는 챗봇을 만들기 위해 필요합니다. 메모리가 없으면 실제 대화가 불가능합니다.